# Exercises - Week 6 - Grid Search - Blackjack

## References

- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html
- https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
- https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
- https://scikit-learn.org/stable/modules/model_evaluation.html
- https://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html
- https://scikit-learn.org/stable/tutorial/statistical_inference/index.html

## Contents
1. Setup 
2. Grid search
3. Datasets

## 1. Setup

Below libraries are loaded, a helper function is defined, and the features dataframe and target series are created for use by the `GridSearchCV` demonstrations that follow.

Load libraries and display version numbers.

In [8]:
import pandas  as pd
import numpy   as np
import sklearn as sk
print('sklearn',sk.__version__)
print('pandas ',pd.__version__)
print('numpy  ',np.__version__)

sklearn 0.20.3
pandas 0.24.2
numpy 1.16.2

These version numbers may not be the most recent or correspond to the documentation you locate via Google.

The `display_pdf` function displays a pandas dataframe using the databricks display function.

In [11]:
def display_pdf(a_pdf):
  display(spark.createDataFrame(a_pdf))

The demonstrations below all use the Boston housing dataset. The following code cell displays its dimensions.

In [13]:
from sklearn.datasets import load_boston
load_boston().data.shape

Out[ 4 ]: (506, 13)

The following cell displayes the first three rows of features.

In [15]:
load_boston().data[ :3]

Out[ 5 ]: 
array([[6.3200e-03, 1.8000e+01, 2.3100e+00, 0.0000e+00, 5.3800e-01,
 6.5750e+00, 6.5200e+01, 4.0900e+00, 1.0000e+00, 2.9600e+02,
 1.5300e+01, 3.9690e+02, 4.9800e+00],
 [2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
 6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
 1.7800e+01, 3.9690e+02, 9.1400e+00],
 [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
 7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
 1.7800e+01, 3.9283e+02, 4.0300e+00]])

Notice that all values are numeric (which makes sense as they are stored in a numpy array).

The following cell displayes the first three values of the target.

In [18]:
load_boston().target[:3]

Out[ 6 ]: array([24. , 21.6, 34.7])

Count the number of missing values in the dataset. (There aren't any.)

In [20]:
import numpy  as np
from sklearn.datasets import load_boston
np.sum(np.isnan(load_boston().data))

Out[ 7 ]: 0

It would be helpful to have some missing values in the dataset. The following code cell creates 20 missing values in `bos_fea_array`.

In [22]:
import pandas as pd
from sklearn.datasets import load_boston
from random           import randint
bos_fea_array = load_boston().data 
for n in range(20): 
  bos_fea_array[randint(0,502), 
                randint(0,12)
               ] = np.nan

The following cell check that we do in fact have 20 missing values.

In [24]:
np.sum(np.isnan(bos_fea_array))

Out[ 9 ]: 20

Define two functions to provide the target series and feature dataframe for the Boston housing dataset, including the additional missing values.

In [26]:
def bos_tgt_ser(): 
  import pandas as pd
  from sklearn import datasets
  return pd.Series(data=datasets.load_boston().target)

def bos_fea_pdf(): 
  import pandas as pd
  from sklearn.datasets import load_boston
  bos_fea_array   = load_boston().data 
  bos_fea_columns = load_boston().feature_names
  for n in range(20): 
    bos_fea_array[randint(0,502), 
                  randint(0,12)
                 ] = np.nan
  return pd.DataFrame(data   =          bos_fea_array,
                      columns=pd.Series(bos_fea_columns).str.lower()
                     )

The commands below display summary details about the target and features, in the next two cells.

In [28]:
bos_fea_pdf().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
crim 505 non-null float64
zn 503 non-null float64
indus 506 non-null float64
chas 502 non-null float64
nox 505 non-null float64
rm 505 non-null float64
age 505 non-null float64
dis 504 non-null float64
rad 504 non-null float64
tax 505 non-null float64
ptratio 505 non-null float64
b 506 non-null float64
lstat 503 non-null float64
dtypes: float64(13)
memory usage: 51.5 KB

The features dataframe only contains `float` values/columns.

In [30]:
bos_tgt_ser()[:3]

Out[ 12 ]: 
0 24.0
1 21.6
2 34.7
dtype: float64

The target series is also of type `float`.

## 2. Grid search

Grid search is a technique for creating models using a range of hyper-parameters and then scoring these predictions made by these models.

First, create a pipeline below which:
1. Imputes missing values
2. Scales values to the `0` to `1` range
3. Performs a linear regression on the data

In [34]:
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression, LinearRegression
from sklearn.preprocessing   import MinMaxScaler
from sklearn.impute          import SimpleImputer

est_pipe = Pipeline([
  ('imp', SimpleImputer()),
  ('sca', MinMaxScaler()),
  ('lrg', LinearRegression())
])

This pipeline will be used in the grid search example below.

Consider the `parameters` dictionary below:
- The first three characters of the keys correspond to names of pipeline steps
- Two underscores follow the name of the pipeline step
- The remainder of the key is the name of an init parameter of the named step
- The value (of the key) is a list of possible values for that named init parameter

In [37]:
parameters = {
  'imp__strategy': ['mean', 'median'],
  'lrg__fit_intercept': [True, False]
}

The following cell performs a grid search. This means that the above pipeline is run:
- for all combinations of parameters listed in the `parameters` dictionary
- in this case, each model created with a combination of hyper-parameters, is fit on the train dataset and predicts on the test dataset of each of three cross validation pairs (see the `cv` parameter)

The keys of the `parameters` dictionary are hyper-parameters of the transformers or final estimator of the pipeline.

The code below:
1. Creates a `GridSearchCV` object, which is an estimator
2. Fits this object to the features and target of the Boston housing dataset

Notice three key parameters of this object:
1. `estimator`: the pipeline, which is a series of transformer objects and a final estimator
2. `param_grid`: the parameters (hyper-parameters of the step objects)
3. `cv`: the number of cross validation pairs

In [39]:
from sklearn.model_selection import GridSearchCV

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                            param_grid=parameters,
                            cv=3, 
                            iid=False,
                            return_train_score=True
                           )
est_grid_obj.fit(bos_fea_pdf(), 
                 bos_tgt_ser()
                )

Out[ 15 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 verbose=0)), ('sca', MinMaxScaler(copy=True, feature_range=(0, 1))), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'lrg__fit_intercept': [True, False], 'imp__strategy': ['mean', 'median']},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
 scoring=None, verbose=0)

The output indicates the steps of the pipeline, including the default parameters for each.

Grid search (`GridSearchCV` in Python) is often used to optimize the choice of hyper-parameters in order that the pipeline (as determined by its hyper-parameters) produces the best prediction score. 

We use `GridSearchCV`  to investigate the effects of the chosen hyper-parameter values on the scoring of the predictions made by the estimator pipeline. This will allow us to better understand a range of estimators and transformers, through the effect of their hyper-parameters on the scoring of their predictions.

In particular, the `GridSearchCV` object contains the `cv_results_` attribute, which contains information on the hyper-parameters and the scoring.

In [43]:
est_grid_obj.best_params_

Out[ 16 ]: {'lrg__fit_intercept': True, 'imp__strategy': 'mean'}

In [44]:
est_grid_obj.cv_results_

Out[ 17 ]: 
{'split0_train_score': array([0.74986759, 0.65912256, 0.75035736, 0.66075196]),
 'std_train_score': array([0.08843996, 0.13322108, 0.08882926, 0.13365675]),
 'params': [{'lrg__fit_intercept': True, 'imp__strategy': 'mean'},
 {'lrg__fit_intercept': False, 'imp__strategy': 'mean'},
 {'lrg__fit_intercept': True, 'imp__strategy': 'median'},
 {'lrg__fit_intercept': False, 'imp__strategy': 'median'}],
 'split1_test_score': array([0.53624436, 0.43885148, 0.53845367, 0.44344075]),
 'std_test_score': array([2.96999718, 6.08046866, 3.0307479 , 6.15657066]),
 'rank_test_score': array([1, 3, 2, 4], dtype=int32),
 'mean_train_score': array([0.76281096, 0.67856238, 0.76276306, 0.67896084]),
 'param_imp__strategy': masked_array(data=['mean', 'mean', 'median', 'median'],
 mask=[False, False, False, False],
 fill_value='?',
 dtype=object),
 'split1_train_score': array([0.66154782, 0.52599132, 0.6607045 , 0.52513119]),
 'mean_fit_time': array([0.01534621, 0.0043029 , 0.00467904, 0.00462612]),
 'split2_train_score': array([0.87701746, 0.85057325, 0.8772273 , 0.85099938]),
 'param_lrg__fit_intercept': masked_array(data=[True, False, True, False],
 mask=[False, False, False, False],
 fill_value='?',
 dtype=object),
 'split0_test_score': array([0.58625867, 0.65741239, 0.59301577, 0.66211032]),
 'std_fit_time': array([1.56081045e-02, 2.67946091e-04, 6.24979137e-05, 6.73512676e-05]),
 'std_score_time': array([1.33632047e-03, 5.61170687e-06, 1.16280500e-05, 1.34099707e-05]),
 'split2_test_score': array([ -5.73891503, -12.34910108, -5.8632788 , -12.50591 ]),
 'mean_test_score': array([-1.538804 , -3.75094573, -1.57726978, -3.80011965]),
 'mean_score_time': array([0.00237028, 0.00142002, 0.0014379 , 0.00142376])}

There is a lot to notice here:
- The keys that start with `param_` contain values from hyper-parameter options in `parameters`
- The keys the end with `_test_score` contain results from the scoring of cross validation train-test pairs

Below this dictionary is converted into a dataframe. The key values are columns in the dataframe.

In [46]:
import pandas as pd
pd.DataFrame(data=est_grid_obj.cv_results_)

Out[ 18 ]: 
 mean_fit_time mean_score_time ... std_test_score std_train_score
0 0.015346 0.002370 ... 2.969997 0.088440
1 0.004303 0.001420 ... 6.080469 0.133221
2 0.004679 0.001438 ... 3.030748 0.088829
3 0.004626 0.001424 ... 6.156571 0.133657

[4 rows x 18 columns]

__Aside:__ what does it mean to summarized one variable in terms of another?

The scoring results can be summarized in terms of the parameter options.
There are three reasons for this:
1. The results are a __dataframe__
2. The parameter options are __categorical variables__ (to be used by the `groupby` method)
3. The score results are __numeric variables__ (to be used by the `agg` method)

The first and last items above are always true, but the second will require some work. We need to ensure that the values of the parameter options are either strings or integers (so they can be used by the `groupby` method).

The example below is a demonstration where this is not the case.

I expect to use this often and so have placed the code above in a function (below).

In [50]:
def est_grid_results_pdf(my_est_grid_obj): 
  import pandas as pd
  return pd.DataFrame(data=my_est_grid_obj.cv_results_)

__Exercise:__ Create a copy of `est_grid_results_pdf` and modify it to only return the parameter options and score results. Run it to produce a pandas dataframe with the reduced set of variable. 

See `Cmd 24` (starting "There is a lot to notice here") for the variables.

Three functions defined above are used below:
- `bos_tgt_ser`: returns the target series from the Boston dataset using the `sklearn.datasets.load_boston` function 
- `bos_fea_pdf`: returns the feature dataframe from the Boston dataset using the `sklearn.datasets.load_boston` function 
- `est_grid_results_pdf`: returns the dataframe of results from a fitted `GridSearchCV` object

The exercises below will focus on the three code cells immediately below.

In [54]:
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression, LinearRegression
from sklearn.preprocessing   import MinMaxScaler
from sklearn.impute          import SimpleImputer

est_pipe = Pipeline([
  ('imp', SimpleImputer()),
  ('sca', MinMaxScaler()),
  ('lrg', LinearRegression())
])

In [55]:
parameters = {
  'imp__strategy': ['mean', 'median'],
  'lrg__fit_intercept': [True, False]
}

from sklearn.model_selection import GridSearchCV

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                        param_grid=parameters,
                        cv=3, 
                        iid=False,
                        return_train_score=False
                       )
est_grid_obj.fit(bos_fea_pdf(), 
                 bos_tgt_ser())

Out[ 21 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 verbose=0)), ('sca', MinMaxScaler(copy=True, feature_range=(0, 1))), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'lrg__fit_intercept': [True, False], 'imp__strategy': ['mean', 'median']},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring=None, verbose=0)

In [56]:
est_grid_results_pdf(est_grid_obj)

Out[ 22 ]: 
 mean_fit_time mean_score_time ... std_score_time std_test_score
0 0.007460 0.001446 ... 0.000029 2.949638
1 0.004180 0.001449 ... 0.000026 6.295999
2 0.004680 0.001438 ... 0.000014 2.917676
3 0.004664 0.001453 ... 0.000033 6.303666

[4 rows x 13 columns]

__Exercise:__ Summarize the output from `est_grid_results_pdf(est_grid_obj)` with the `groupby` and `agg` methods.

I understand that there are only four cases and that you can do this visually, but use these methods to learn the technique.

The following two lines of code show how to use the groupby and agg functions.  There is not much of interest going on with the code or much worth noting here.

In [59]:
x = est_grid_results_pdf(est_grid_obj).groupby(est_grid_results_pdf(est_grid_obj)['mean_fit_time'])

In [60]:
x1 = est_grid_results_pdf(est_grid_obj).groupby(est_grid_results_pdf(est_grid_obj)['std_score_time'])

__Exercise:__ copy the three code cells above and modify as below:
- Add `SelectKBest()` to the pipeline in the third position, just before the `LinearRegression` 
- Run `GridSearchCV` on this pipeline (with the same parameters as before)
- Check that reasonable looking results are produced by the `est_grid_results_pdf` function

This initial block of text serves to explain oour understanidng of the grid search method and works through the three code block segemnts. The first code block sets up the. creation of an estimator pipeline which includes the imported things from sklearn. The pipeline will unfctiono as an estimator, thus the need for us to put the SelectKBest above ogisitc regression, because had we not than there would have been issues in running. any predictions.  The next code block defines the parameters to be included in the grid search and under which the grid search will be conducted. For our purposes, the inital parameters are the imputing strtaegy being mean or median, and the logistiic regression intercept being true or false (indicating weather the log regression funciton needs to enter the origin).  After this, we load in the. Grid Search function, and create a grid search object, which takes the arguments estimator, parameter grid, and number of cross validations to be performed.  We also choose to specify iid to be false, and in our case return of the train score is set to false.  We finally call the object created to be fit onto the boston features dataframe and the boston target series - mimiking what will be expected out of us later on in the project.  We then call the pdf funtion defined above to show the results of the object we created (est_grid_object). Future text will refer nack to this cell as "original explanation".

In [63]:
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression, LinearRegression
from sklearn.preprocessing   import MinMaxScaler
from sklearn.impute          import SimpleImputer
from sklearn.feature_selection import SelectKBest

est_pipe = Pipeline([
  ('imp', SimpleImputer()),
  ('sca', MinMaxScaler()),
  ('kbest', SelectKBest()),
  ('lrg', LinearRegression())
])

In [64]:
parameters = {
  'imp__strategy': ['mean', 'median'],
  'lrg__fit_intercept': [True, False]
}

from sklearn.model_selection import GridSearchCV

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                        param_grid=parameters,
                        cv=3, 
                        iid=False,
                        return_train_score=False
                       )
est_grid_obj.fit(bos_fea_pdf(), 
                 bos_tgt_ser())

Out[ 26 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 verbose=0)), ('sca', MinMaxScaler(copy=True, feature_range=(0, 1))), ('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x7f1f8aac3f28>)), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'lrg__fit_intercept': [True, False], 'imp__strategy': ['mean', 'median']},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring=None, verbose=0)

In [65]:
est_grid_results_pdf(est_grid_obj)

Out[ 27 ]: 
 mean_fit_time mean_score_time ... std_score_time std_test_score
0 0.029098 0.003007 ... 0.000025 0.700250
1 0.030743 0.002571 ... 0.000640 2.016271
2 0.016862 0.001699 ... 0.000010 0.734699
3 0.018962 0.001951 ... 0.000034 2.071746

[4 rows x 13 columns]

__Exercise:__ copy the three code cells from the previous exercise and modify as below:
- Add a key-value pair to the `parameters` dictionary for the `k` parameter of the `SelectKBest` class/object with value `[13,10,7,4]`
- Run `GridSearchCV` on the pipeline with the new `parameters` dictionary
- Use the `est_grid_results_pdf` function modified in a previous exercise
- Check that reasonable looking results are produced and notice the column for the new parameter

This text block will discuss the additional code from the "original explanation".  The second line of code now includes a new parameter to be tested, the SelectKBest with differing values as specified in the exercise.  This means that the total number of results in or Grid Serach will be 16 (4x2x2) and mean sthat we will hopefully find a better answer when compared to previously run segments.

In [68]:
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression, LinearRegression
from sklearn.preprocessing   import MinMaxScaler
from sklearn.impute          import SimpleImputer
from sklearn.feature_selection import SelectKBest

est_pipe = Pipeline([
  ('imp', SimpleImputer()),
  ('sca', MinMaxScaler()),
  ('kbest', SelectKBest()),
  ('lrg', LinearRegression())
])

Note the presence of the kbest param.

In [70]:
parameters = {
  'imp__strategy': ['mean', 'median'],
  'lrg__fit_intercept': [True, False],
  'kbest__k': [13,10,7,4]
}

from sklearn.model_selection import GridSearchCV

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                        param_grid=parameters,
                        cv=3, 
                        iid=False,
                        return_train_score=False
                       )
est_grid_obj.fit(bos_fea_pdf(), 
                 bos_tgt_ser())

Out[ 29 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 verbose=0)), ('sca', MinMaxScaler(copy=True, feature_range=(0, 1))), ('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x7f1f8aac3f28>)), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'lrg__fit_intercept': [True, False], 'imp__strategy': ['mean', 'median'], 'kbest__k': [13, 10, 7, 4]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring=None, verbose=0)

In [71]:
est_grid_obj.best_params_

Out[ 30 ]: {'lrg__fit_intercept': True, 'imp__strategy': 'median', 'kbest__k': 7}

Note there are 16 rows representing the 16 different grid boxes that were ran for the parameters given.

In [73]:
est_grid_results_pdf(est_grid_obj)

Out[ 31 ]: 
 mean_fit_time mean_score_time ... std_score_time std_test_score
0 0.023445 0.002147 ... 6.229977e-04 2.910763
1 0.016606 0.001700 ... 1.805416e-05 5.437493
2 0.016789 0.001682 ... 1.197063e-05 0.880686
3 0.016380 0.001692 ... 1.913399e-05 2.637301
4 0.016813 0.001684 ... 2.103975e-05 0.316890
5 0.016511 0.001711 ... 8.485379e-07 1.052552
6 0.016758 0.001699 ... 1.214037e-05 1.139704
7 0.016696 0.001704 ... 1.767516e-05 1.151538
8 0.016984 0.001709 ... 5.106093e-06 2.825010
9 0.016827 0.001768 ... 3.693171e-05 5.212288
10 0.017157 0.001685 ... 2.192097e-05 0.822292
11 0.016978 0.001676 ... 5.437942e-06 2.473927
12 0.017026 0.001713 ... 3.327873e-05 0.309983
13 0.016829 0.001709 ... 1.349963e-05 1.047450
14 0.016680 0.001727 ... 5.866077e-05 1.077180
15 0.017115 0.001719 ... 3.066025e-05 1.124270

[16 rows x 14 columns]

__Exercise:__ copy the three code cells from the previous exercise and modify as below:
- Import `f_regression` and `mutual_info_regression` from `sklearn.feature_selection` (first line of first code cell)
- Add the `score_func` parameter of `SelectKBest` to the `parameters` dictionary with value `[f_regression,mutual_info_regression]`
- Run `GridSearchCV` with this modified `parameters` dictionary
- Check the column for the `score_func` parameter in the resulting dataframe. Notice that it is not a string or number.

This text block will discuss the additional code from the "original explanation".  We add a new parameter to be considered from the kbest section.  This parameter specifies two different methodologies for the code to choose between, either f_regression or mutual_info_regression.  Note that the grid search was able to run before this because kbest had a default parameter for scoring originally.  Because of the increased parameters there will now be 32 grids or rows to consider when looking at our grid search results.

In [76]:
from sklearn.pipeline        import Pipeline
from sklearn.linear_model    import LogisticRegression, LinearRegression
from sklearn.preprocessing   import MinMaxScaler
from sklearn.impute          import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

est_pipe = Pipeline([
  ('imp', SimpleImputer()),
  ('sca', MinMaxScaler()),
  ('kbest', SelectKBest()),
  ('lrg', LinearRegression())
])

Note presence of the score function defined to test f_regression and mutual_info_regression

In [78]:
parameters = {
  'imp__strategy': ['mean', 'median'],
  'lrg__fit_intercept': [True, False],
  'kbest__k': [13,10,7,4],
  'kbest__score_func': [f_regression, mutual_info_regression]
}

from sklearn.model_selection import GridSearchCV

est_grid_obj = GridSearchCV(estimator=est_pipe, 
                        param_grid=parameters,
                        cv=3, 
                        iid=False,
                        return_train_score=False
                       )
est_grid_obj.fit(bos_fea_pdf(), 
                 bos_tgt_ser())

Out[ 33 ]: 
GridSearchCV(cv=3, error_score='raise-deprecating',
 estimator=Pipeline(memory=None,
 steps=[('imp', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
 verbose=0)), ('sca', MinMaxScaler(copy=True, feature_range=(0, 1))), ('kbest', SelectKBest(k=10, score_func=<function f_classif at 0x7f1f8aac3f28>)), ('lrg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
 normalize=False))]),
 fit_params=None, iid=False, n_jobs=None,
 param_grid={'kbest__score_func': [<function f_regression at 0x7f1f8aad5158>, <function mutual_info_regression at 0x7f1f8a5e1ea0>], 'lrg__fit_intercept': [True, False], 'imp__strategy': ['mean', 'median'], 'kbest__k': [13, 10, 7, 4]},
 pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
 scoring=None, verbose=0)

In [79]:
est_grid_obj.best_params_

Out[ 34 ]: 
{'kbest__score_func': <function sklearn.feature_selection.univariate_selection.f_regression>,
 'lrg__fit_intercept': True,
 'imp__strategy': 'median',
 'kbest__k': 4}

We note 32 potential rows to choose from when looking at the results.

In [81]:
est_grid_results_pdf(est_grid_obj)

Out[ 35 ]: 
 mean_fit_time mean_score_time ... std_score_time std_test_score
0 0.005070 0.001553 ... 2.019424e-05 2.111515
1 0.004952 0.001554 ... 2.148885e-05 5.221483
2 0.066660 0.001680 ... 4.836758e-06 2.111515
3 0.062445 0.001670 ... 1.795700e-05 5.221483
4 0.004940 0.001546 ... 7.840875e-06 1.401716
5 0.004896 0.001568 ... 2.245020e-06 3.980021
6 0.069254 0.001712 ... 3.414208e-05 1.022314
7 0.063256 0.001712 ... 8.693474e-06 0.762212
8 0.004913 0.001556 ... 1.454462e-05 0.427734
9 0.004825 0.001561 ... 1.693648e-05 0.917861
10 0.065489 0.001703 ... 8.305579e-06 0.374454
11 0.062385 0.001686 ... 2.725351e-06 1.078838
12 0.004917 0.001545 ... 1.250480e-05 0.389971
13 0.004997 0.001579 ... 1.480799e-05 0.974968
14 0.064369 0.001818 ... 1.456221e-04 0.398291
15 0.062607 0.001704 ... 1.749234e-05 1.751331
16 0.005413 0.001676 ... 1.195295e-04 2.091197
17 0.005336 0.001590 ... 6.257699e-07 5.323450
18 0.063344 0.001694 ... 2.238625e-05 2.091197
19 0.063517 0.001693 ... 1.461220e-05 5.323450
20 0.005391 0.001591 ... 4.483002e-06 1.418517
21 0.005296 0.001542 ... 6.432885e-06 4.089392
22 0.063251 0.001699 ... 1.854598e-05 1.015126
23 0.064278 0.001697 ... 1.650550e-05 0.806227
24 0.005351 0.001596 ... 7.377719e-06 0.426264
25 0.005377 0.001662 ... 8.536939e-05 0.919545
26 0.063400 0.001689 ... 2.146915e-05 0.373777
27 0.069586 0.001829 ... 1.905243e-04 1.081083
28 0.005393 0.001577 ... 3.570092e-06 0.389015
29 0.005258 0.001550 ... 3.316980e-06 0.975520
30 0.062913 0.001778 ... 1.357780e-04 0.389387
31 0.068377 0.002100 ... 4.643371e-04 1.752887

[32 rows x 15 columns]

__Exercise:__ create a wrapper class called `SelectKBestWrap` for `SelectKBest`:
- Create a `SelectKBest` object in the `init` method
- Create an init parameter called `score_func_str` 
- When this parameter is `'f_regression'` then pass `f_regression` to `SelectKBest`
- When this parameter is `'mutual_info_regression'` then pass `mutual_info_regression` to `SelectKBest`
- You will need to create `fit` and `transform` methods (see examples from a previous week) which return results 
- Test this function on the Boston features dataframe and the target series

The below code creates teh wrapper class for the SelectKBEst functionality.  We coded into the int function two if tests to dtermine what the input was for the given results.  When writting this code it is important to specify this, because it will allow us to perform the analyses needed without typing them in or knowing them - this could proove useful in machine learning, because we will now be able to take the output from our grid search, and refer to i as the input needed for the wrapper class and conduct further analyses.

In [85]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.base import BaseEstimator, TransformerMixin

class SelectKBestWrap(BaseEstimator, TransformerMixin):
  def __init__ (self, score_func_str):
    self.score_func_str = score_func_str
    if self.score_func_str == 'f_regression':
      abc = f_regression
    if self.score_func_str == 'mutual_info_regression':
      abc = mutual_info_regression
    self.SK=SelectKBest(score_func = abc,k=6)
  def fit(self,X,y):
    self.SK.fit(X,y)
    return self
  def transform(self, X):
    X=self.SK.transform(X)
    return X

We see below the code functioning as expected, by replacing the 'na' portions of the features df with the mean (simple form of imputing) we are then able to use our transformer pipeline defined above. We note that the below output is working because there are a similar amount of rows to the original dataset, and the number of columns fitrs with what was specified in the SelectKBest wrapper class.

In [87]:
raw1=bos_fea_pdf()
raw2=raw1.fillna(raw1.iloc[0,:].mean())
a=SelectKBestWrap('f_regression').fit_transform(raw2, bos_tgt_ser())
a.shape

Out[ 37 ]: (506, 6)

## 3. Datasets

This section contains the location and a few notes about the datasets from a previous week.

Display the paths of the three files in our dataset.

In [91]:
%sh ls -hot /dbfs/mnt/group-ma707/data/*

-rw-r--r-- 1 root 259K Jan 29 17:44 /dbfs/mnt/group-ma707/data/5tc_plus_ind_vars.csv
-rw-r--r-- 1 root 12M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_coal.csv
-rw-r--r-- 1 root 11M Jan 29 17:44 /dbfs/mnt/group-ma707/data/mining_com_iron_ore.csv

__Note:__ we will create, from the above files, at least three dataframes (with features and target). Each is described below.

From only the 5TC dataset: 
- target will be `BCI`
- features will include lagged versions of the other columns 
- features will include date and time components (hour, day of week, etc.)
- features may include external time series

From only the _mining_ dataset(s):
- the target may be one or more tags (from the `tags` variable)
- features would be words present in the `content` or `title` variables

From the 5TC and _mining_ dataset(s): 
- target will be `BCI` (from 5TC dataframe)
- include all features from either of the above dataframes
- the dataframes would need to be joined by either:
    1. aggregating the features from the _mining_ dataframe (by date)
    1. spreading the 5TC dataframe onto the _mining_ dataframe (duplicating 5TC rows)

__The End__